# LSE-UKHSA Systematic Review Screening Project

**Documentation & Pipeline Overview**

---

## 1. Project Objective

This project evaluates whether **Large Language Models (LLMs)** can assist in the **title/abstract screening phase** of systematic reviews.

### Research Question
> *Given a Cochrane review's title and abstract (which defines the inclusion criteria), can an LLM correctly determine whether a candidate study should be INCLUDED or EXCLUDED?*

### Why This Matters
- Systematic review screening is **time-consuming** (often 1000s of papers per review)
- **Human reviewers** typically screen each abstract in 30 seconds to 2 minutes
- Automating or semi-automating this could **dramatically reduce workload**
- We use Cochrane's rigorous human decisions as **ground truth** for evaluation

---

## 2. Pipeline Architecture Diagram

In [75]:
# Pipeline Architecture Diagram - Staggered Layout with Outputs on Right
import plotly.graph_objects as go

fig = go.Figure()

# Colors
API_BG = 'rgba(173, 216, 230, 0.3)'  # Light blue background
OUT_BG = 'rgba(255, 235, 153, 0.4)'  # Light yellow background
API_COLOR = '#5DADE2'    # Teal/Blue for APIs
NB_COLOR = '#808B96'     # Grey for notebooks
OUT_COLOR = '#F4D03F'    # Yellow/Gold for outputs
ARROW_COLOR = '#E74C3C'  # Red/coral for arrows
DARK = '#2C3E50'

# Background regions
fig.add_shape(type="rect", x0=0, y0=8.5, x1=11.5, y1=10.2,
              fillcolor=API_BG, line=dict(width=0), layer="below")
fig.add_shape(type="rect", x0=9.5, y0=0, x1=11.5, y1=8.5,
              fillcolor=OUT_BG, line=dict(width=0), layer="below")

# Section labels
fig.add_annotation(x=5.75, y=9.9, text="<b>Sources / APIs / Applications</b>", showarrow=False,
                   font=dict(size=13, color=API_COLOR))
fig.add_annotation(x=10.5, y=8.2, text="<b>Outputs</b>", showarrow=False,
                   font=dict(size=13, color='#B7950B'))
fig.add_annotation(x=4.5, y=7.8, text="<b>Notebooks</b>", showarrow=False,
                   font=dict(size=13, color=NB_COLOR))

def add_box(x, y, w, h, text, color, font_color='white', font_size=10):
    fig.add_shape(type="rect", x0=x-w/2, y0=y-h/2, x1=x+w/2, y1=y+h/2,
                  fillcolor=color, line=dict(color=DARK, width=1.5),
                  layer="below")
    fig.add_annotation(x=x, y=y, text=text, showarrow=False,
                       font=dict(size=font_size, color=font_color), align="center")

def add_arrow(x0, y0, x1, y1, color=ARROW_COLOR):
    fig.add_annotation(x=x1, y=y1, ax=x0, ay=y0, xref="x", yref="y",
                       axref="x", ayref="y", showarrow=True,
                       arrowhead=2, arrowsize=1, arrowwidth=1.5, arrowcolor=color)

# ===== APIs (top row) =====
add_box(1.5, 9.3, 1.6, 0.7, "PubMed API<br>(NCBI Entrez)", API_COLOR, 'white', 9)
add_box(3.5, 9.3, 1.6, 0.7, "Wiley TDM<br>API", API_COLOR, 'white', 9)
add_box(5.5, 9.3, 1.6, 0.7, "CrossRef<br>API", API_COLOR, 'white', 9)
add_box(7.5, 9.3, 1.6, 0.7, "PubMed API<br>(NCBI Entrez)", API_COLOR, 'white', 9)
add_box(9.5, 9.3, 1.6, 0.7, "Ollama<br>(Local LLMs)", API_COLOR, 'white', 9)

# ===== Notebooks (staggered diagonal) =====
add_box(2.5, 7.2, 2.2, 0.8, "00_obtain_cochrane<br>_abstracts.ipynb", NB_COLOR, 'white', 9)
add_box(4, 5.8, 2.2, 0.8, "02_fetch_cochrane<br>_pdfs.ipynb", NB_COLOR, 'white', 9)
add_box(5.5, 4.4, 2.4, 0.8, "03_extract_metadata<br>_and_references.ipynb", NB_COLOR, 'white', 8)
add_box(7, 3, 2.4, 0.8, "04_fetch_referenced<br>_abstracts.ipynb", NB_COLOR, 'white', 9)
add_box(5.5, 1.6, 2.2, 0.8, "05_build_ground<br>_truth.ipynb", NB_COLOR, 'white', 9)
add_box(8, 0.4, 2, 0.7, "06_evaluate<br>_llms.ipynb", NB_COLOR, 'white', 9)

# ===== Outputs (right column) =====
add_box(10.5, 7.2, 1.8, 0.7, "cochrane_pubmed<br>_abstracts.csv<br>(17,298 reviews)", OUT_COLOR, DARK, 8)
add_box(10.5, 5.8, 1.8, 0.7, "cochrane_pdfs/<br>(16,588 PDFs)", OUT_COLOR, DARK, 8)
add_box(10.5, 4.4, 1.8, 0.7, "categorized<br>_references.csv<br>(629K refs)", OUT_COLOR, DARK, 8)
add_box(10.5, 3, 1.8, 0.7, "referenced_paper<br>_abstracts.csv<br>(47,518 papers)", OUT_COLOR, DARK, 8)
add_box(10.5, 1.6, 1.8, 0.7, "ground_truth<br>_validation.csv<br>(41,692 records)", OUT_COLOR, DARK, 8)
add_box(10.5, 0.4, 1.8, 0.6, "results/eval_*.csv", OUT_COLOR, DARK, 9)

# ===== Arrows: APIs → Notebooks =====
add_arrow(1.5, 8.95, 2.5, 7.6, DARK)    # PubMed → 00
add_arrow(3.5, 8.95, 4, 6.2, DARK)      # Wiley → 02
add_arrow(5.5, 8.95, 7, 3.4, DARK)      # CrossRef → 04
add_arrow(7.5, 8.95, 7, 3.4, DARK)      # PubMed → 04
add_arrow(9.5, 8.95, 8, 0.75, DARK)     # Ollama → 06

# ===== Arrows: Notebooks → Outputs =====
add_arrow(3.6, 7.2, 9.6, 7.2)           # 00 → abstracts.csv
add_arrow(5.1, 5.8, 9.6, 5.8)           # 02 → pdfs
add_arrow(6.7, 4.4, 9.6, 4.4)           # 03 → refs
add_arrow(8.2, 3, 9.6, 3)               # 04 → papers
add_arrow(6.6, 1.6, 9.6, 1.6)           # 05 → GT
add_arrow(9, 0.4, 9.6, 0.4)             # 06 → results

# ===== Arrows: Notebook flow (diagonal) =====
add_arrow(3.6, 7.0, 4, 6.2)             # 00 → 02
add_arrow(5.1, 5.6, 5.5, 4.8)           # 02 → 03
add_arrow(6.7, 4.2, 7, 3.4)             # 03 → 04
add_arrow(7, 2.6, 5.5, 2)               # 04 → 05
add_arrow(6.6, 1.4, 8, 0.75)            # 05 → 06

fig.update_layout(
    title=dict(text="<b>LSE-UKHSA Systematic Review Screening Pipeline</b>",
               font=dict(size=16, color=DARK, family="Arial"), x=0.5, y=0.98),
    xaxis=dict(range=[0, 12], showgrid=False, zeroline=False, showticklabels=False, fixedrange=True),
    yaxis=dict(range=[-0.3, 10.5], showgrid=False, zeroline=False, showticklabels=False, fixedrange=True),
    plot_bgcolor='white',
    paper_bgcolor='white',
    width=950,
    height=800,
    margin=dict(l=10, r=10, t=50, b=10)
)

fig.show()

# Export to HTML
html_path = project_root / 'pipeline_workflow_diagram.html' if 'project_root' in dir() else Path.cwd().parent / 'pipeline_workflow_diagram.html'
fig.write_html(str(html_path), include_plotlyjs='cdn')
print(f"\n✓ Exported workflow diagram to: {html_path}")


✓ Exported workflow diagram to: c:\Users\juanx\Documents\LSE-UKHSA Project\pipeline_workflow_diagram.html


---

## 3. Data Pipeline Statistics

The following code calculates and displays all key statistics from the actual data files.

In [64]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# Setup paths
notebook_dir = Path.cwd()
project_root = notebook_dir.parent if (notebook_dir / 'Data').exists() == False else notebook_dir
DATA_DIR = project_root / 'Data'

print("="*70)
print("STAGE 1: COCHRANE REVIEW ABSTRACTS")
print("="*70)
print("Source: PubMed API (NCBI Entrez)")
print("Notebook: 00_obtain_cochrane_abstracts.ipynb")
print("-"*70)

abstracts = pd.read_csv(DATA_DIR / 'cochrane_pubmed_abstracts.csv', dtype=str)
print(f"Total Cochrane reviews fetched:    {len(abstracts):,}")
print(f"Reviews with DOI:                  {abstracts['doi'].notna().sum():,}")
print(f"Reviews with abstract text:        {abstracts['abstract'].notna().sum():,}")

STAGE 1: COCHRANE REVIEW ABSTRACTS
Source: PubMed API (NCBI Entrez)
Notebook: 00_obtain_cochrane_abstracts.ipynb
----------------------------------------------------------------------
Total Cochrane reviews fetched:    17,298
Reviews with DOI:                  17,297
Reviews with abstract text:        17,112


In [65]:
print("="*70)
print("STAGE 2: PDF DOWNLOADS")
print("="*70)
print("Source: Wiley TDM API")
print("Notebook: 02_fetch_cochrane_pdfs.ipynb")
print("-"*70)

pdf_dir = DATA_DIR / 'cochrane_pdfs'
if pdf_dir.exists():
    pdf_count = len(list(pdf_dir.glob('*.pdf')))
    print(f"Total PDFs downloaded:             {pdf_count:,}")
    print(f"Download rate:                     {pdf_count/len(abstracts)*100:.1f}% of reviews")
else:
    print("PDF directory not found")

STAGE 2: PDF DOWNLOADS
Source: Wiley TDM API
Notebook: 02_fetch_cochrane_pdfs.ipynb
----------------------------------------------------------------------
Total PDFs downloaded:             16,588
Download rate:                     95.9% of reviews


In [66]:
print("="*70)
print("STAGE 3: REFERENCE EXTRACTION FROM PDFs")
print("="*70)
print("Source: Local PDF processing (PyMuPDF)")
print("Notebook: 03_extract_metadata_and_references.ipynb")
print("-"*70)

refs = pd.read_csv(DATA_DIR / 'categorized_references.csv', dtype=str, low_memory=False)
print(f"Total references extracted:        {len(refs):,}")
print(f"  - Included studies:              {(refs['category']=='included').sum():,}")
print(f"  - Excluded studies:              {(refs['category']=='excluded').sum():,}")
print(f"  - Awaiting classification:       {(refs['category']=='awaiting').sum():,}")
print(f"  - Ongoing studies:               {(refs['category']=='ongoing').sum():,}")
print("-"*70)
print(f"References with DOI extracted:     {refs['ref_doi'].notna().sum():,} ({refs['ref_doi'].notna().mean()*100:.1f}%)")
print(f"References with PMID extracted:    {refs['pmid'].notna().sum():,} ({refs['pmid'].notna().mean()*100:.1f}%)")

STAGE 3: REFERENCE EXTRACTION FROM PDFs
Source: Local PDF processing (PyMuPDF)
Notebook: 03_extract_metadata_and_references.ipynb
----------------------------------------------------------------------
Total references extracted:        629,561
  - Included studies:              210,887
  - Excluded studies:              387,426
  - Awaiting classification:       22,630
  - Ongoing studies:               8,618
----------------------------------------------------------------------
References with DOI extracted:     31,538 (5.0%)
References with PMID extracted:    46,483 (7.4%)


In [67]:
print("="*70)
print("STAGE 4: REFERENCE MATCHING & ABSTRACT FETCHING")
print("="*70)
print("Source: CrossRef API + PubMed API")
print("Notebook: 04_fetch_referenced_abstracts.ipynb")
print("-"*70)

papers = pd.read_csv(DATA_DIR / 'referenced_paper_abstracts.csv', dtype=str, low_memory=False)
print(f"Total references successfully matched: {len(papers):,}")
print(f"Match rate from all references:        {len(papers)/len(refs)*100:.1f}%")
print("-"*70)
print("Match method breakdown:")
for method, count in papers['match_method'].value_counts().items():
    print(f"  - {method}:  {count:,} ({count/len(papers)*100:.1f}%)")
print("-"*70)
print(f"Papers with abstract retrieved:        {papers['abstract'].notna().sum():,} ({papers['abstract'].notna().mean()*100:.1f}%)")
print("-"*70)
print("Matched papers by category:")
for cat, count in papers['category'].value_counts().items():
    print(f"  - {cat}: {count:,}")

STAGE 4: REFERENCE MATCHING & ABSTRACT FETCHING
Source: CrossRef API + PubMed API
Notebook: 04_fetch_referenced_abstracts.ipynb
----------------------------------------------------------------------
Total references successfully matched: 47,518
Match rate from all references:        7.5%
----------------------------------------------------------------------
Match method breakdown:
  - crossref:  47,121 (99.2%)
  - doi_direct:  397 (0.8%)
----------------------------------------------------------------------
Papers with abstract retrieved:        42,873 (90.2%)
----------------------------------------------------------------------
Matched papers by category:
  - excluded: 30,384
  - included: 15,921
  - awaiting: 719
  - ongoing: 494


In [68]:
print("="*70)
print("STAGE 5: GROUND TRUTH VALIDATION DATASET")
print("="*70)
print("Source: Joins abstracts + matched papers + categories")
print("Notebook: 05_build_ground_truth.ipynb")
print("-"*70)

gt = pd.read_csv(DATA_DIR / 'ground_truth_validation_dataset.csv', dtype=str, low_memory=False)
gt['label'] = gt['label'].astype(int)

print(f"Total validation records:          {len(gt):,}")
print(f"Unique Cochrane reviews:           {gt['review_doi'].nunique():,}")
print("-"*70)
print("Label distribution:")
print(f"  - INCLUDE (label=1):             {(gt['label']==1).sum():,} ({(gt['label']==1).mean()*100:.1f}%)")
print(f"  - EXCLUDE (label=0):             {(gt['label']==0).sum():,} ({(gt['label']==0).mean()*100:.1f}%)")
print("-"*70)
print("Records by Cochrane Group:")
for group, count in gt['cochrane_group'].value_counts().items():
    include_rate = (gt[gt['cochrane_group']==group]['label']==1).mean()*100
    print(f"  - {group}: {count:,} ({include_rate:.0f}% include)")

STAGE 5: GROUND TRUTH VALIDATION DATASET
Source: Joins abstracts + matched papers + categories
Notebook: 05_build_ground_truth.ipynb
----------------------------------------------------------------------
Total validation records:          41,692
Unique Cochrane reviews:           1,228
----------------------------------------------------------------------
Label distribution:
  - INCLUDE (label=1):             14,738 (35.3%)
  - EXCLUDE (label=0):             26,954 (64.7%)
----------------------------------------------------------------------
Records by Cochrane Group:
  - Acute Respiratory Infections: 11,455 (36% include)
  - Tobacco Addiction: 10,198 (43% include)
  - Infectious Diseases: 8,516 (33% include)
  - Drugs and Alcohol: 6,754 (34% include)
  - Public Health: 4,089 (21% include)
  - STI: 680 (43% include)


In [69]:
print("="*70)
print("EVALUATION SUBSET: PUBLIC HEALTH")
print("="*70)

ph = gt[gt['cochrane_group'] == 'Public Health']
print(f"Public Health records:             {len(ph):,}")
print(f"  - INCLUDE:                       {(ph['label']==1).sum():,} ({(ph['label']==1).mean()*100:.1f}%)")
print(f"  - EXCLUDE:                       {(ph['label']==0).sum():,} ({(ph['label']==0).mean()*100:.1f}%)")
print("-"*70)
print("This subset is used for LLM evaluation due to:")
print("  1. Domain relevance to UKHSA")
print("  2. Manageable size for local inference")
print("  3. Representative class imbalance (~80% exclude)")

EVALUATION SUBSET: PUBLIC HEALTH
Public Health records:             4,089
  - INCLUDE:                       848 (20.7%)
  - EXCLUDE:                       3,241 (79.3%)
----------------------------------------------------------------------
This subset is used for LLM evaluation due to:
  1. Domain relevance to UKHSA
  2. Manageable size for local inference
  3. Representative class imbalance (~80% exclude)


---

## 4. Pipeline Flow Visualization

In [70]:
# Sankey-style funnel visualization
import plotly.graph_objects as go

# Data from pipeline
stages = [
    'Cochrane Reviews\n(PubMed)',
    'PDFs Downloaded\n(Wiley TDM)',
    'References Extracted\n(PyMuPDF)',
    'INCLUDE + EXCLUDE\nReferences',
    'Matched to PubMed\n(CrossRef)',
    'With Abstracts',
    'Validation Dataset'
]

values = [17298, 16588, 629561, 598313, 47518, 42873, 41692]

fig = go.Figure(go.Funnel(
    y = stages,
    x = values,
    textposition = "inside",
    textinfo = "value+percent previous",
    opacity = 0.85,
    marker = dict(
        color = ['#5DADE2', '#5DADE2', '#F4D03F', '#F4D03F', '#F4D03F', '#F4D03F', '#27AE60'],
        line = dict(width = 2, color = '#333333')
    ),
    connector = dict(line=dict(color="#C0392B", width=2))
))

fig.update_layout(
    title=dict(text="Data Pipeline Funnel: From Reviews to Validation Dataset", font=dict(size=18)),
    font=dict(size=12),
    height=600,
    width=800
)

fig.show()

---

## 5. Reference Matching Deep Dive

A key challenge in this pipeline is **matching citation strings to PubMed records**. Here's the detailed breakdown:

In [71]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Reference matching statistics
total_refs = 629561
included_excluded = 598313  # included + excluded categories
with_doi_or_pmid = 31538 + 46483 - 10000  # approximate overlap
matched_via_crossref = 47121
matched_via_doi_direct = 397
total_matched = 47518
with_abstract = 42873

# Create matching funnel
fig = make_subplots(rows=1, cols=2, 
                    specs=[[{"type": "domain"}, {"type": "xy"}]],
                    subplot_titles=('Match Method Distribution', 'Matching Success by Stage'))

# Pie chart for match methods
fig.add_trace(go.Pie(
    labels=['CrossRef Bibliographic Search', 'Direct DOI Extraction'],
    values=[47121, 397],
    hole=0.4,
    marker_colors=['#3498DB', '#E74C3C']
), row=1, col=1)

# Bar chart for matching stages
categories = ['Total\nReferences', 'Inc+Exc\nCategories', 'Matched to\nPubMed', 'With\nAbstracts']
values = [629561, 598313, 47518, 42873]
colors = ['#E8E8E8', '#B3B3B3', '#5DADE2', '#27AE60']

fig.add_trace(go.Bar(
    x=categories,
    y=values,
    marker_color=colors,
    text=[f'{v:,}' for v in values],
    textposition='outside'
), row=1, col=2)

fig.update_layout(
    title='Reference Matching Analysis',
    height=450,
    showlegend=True
)

fig.show()

print("\n" + "="*70)
print("REFERENCE MATCHING SUMMARY")
print("="*70)
print(f"Total references extracted from PDFs:     {total_refs:,}")
print(f"References in INCLUDE/EXCLUDE categories: {included_excluded:,} ({included_excluded/total_refs*100:.1f}%)")
print(f"Successfully matched to PubMed:           {total_matched:,} ({total_matched/included_excluded*100:.1f}%)")
print(f"  └─ Via CrossRef search:                 {matched_via_crossref:,} ({matched_via_crossref/total_matched*100:.1f}%)")
print(f"  └─ Via direct DOI extraction:           {matched_via_doi_direct:,} ({matched_via_doi_direct/total_matched*100:.1f}%)")
print(f"With abstract text available:             {with_abstract:,} ({with_abstract/total_matched*100:.1f}%)")
print("="*70)
print(f"\nNOTE: {included_excluded - total_matched:,} references ({(included_excluded-total_matched)/included_excluded*100:.1f}%) could not be matched.")
print("Common reasons: Conference papers, books, non-English publications,")
print("grey literature, or incomplete citation information in PDFs.")


REFERENCE MATCHING SUMMARY
Total references extracted from PDFs:     629,561
References in INCLUDE/EXCLUDE categories: 598,313 (95.0%)
Successfully matched to PubMed:           47,518 (7.9%)
  └─ Via CrossRef search:                 47,121 (99.2%)
  └─ Via direct DOI extraction:           397 (0.8%)
With abstract text available:             42,873 (90.2%)

NOTE: 550,795 references (92.1%) could not be matched.
Common reasons: Conference papers, books, non-English publications,
grey literature, or incomplete citation information in PDFs.


---

## 6. Validation Dataset Class Distribution

In [72]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load ground truth
gt = pd.read_csv(DATA_DIR / 'ground_truth_validation_dataset.csv', dtype=str)
gt['label'] = gt['label'].astype(int)

# Group statistics
group_stats = gt.groupby('cochrane_group').agg(
    total=('label', 'count'),
    include=('label', 'sum'),
).reset_index()
group_stats['exclude'] = group_stats['total'] - group_stats['include']
group_stats['include_rate'] = group_stats['include'] / group_stats['total'] * 100
group_stats = group_stats.sort_values('total', ascending=True)

# Create stacked horizontal bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    y=group_stats['cochrane_group'],
    x=group_stats['include'],
    name='INCLUDE',
    orientation='h',
    marker_color='#27AE60',
    text=[f"{r:.0f}%" for r in group_stats['include_rate']],
    textposition='inside'
))

fig.add_trace(go.Bar(
    y=group_stats['cochrane_group'],
    x=group_stats['exclude'],
    name='EXCLUDE',
    orientation='h',
    marker_color='#E74C3C'
))

fig.update_layout(
    barmode='stack',
    title='Validation Dataset: Class Distribution by Cochrane Group',
    xaxis_title='Number of Records',
    yaxis_title='',
    height=400,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()

# Summary table
print("\nValidation Dataset Summary:")
print(group_stats[['cochrane_group', 'total', 'include', 'exclude', 'include_rate']].to_string(index=False))


Validation Dataset Summary:
              cochrane_group  total  include  exclude  include_rate
                         STI    680      291      389     42.794118
               Public Health   4089      848     3241     20.738567
           Drugs and Alcohol   6754     2302     4452     34.083506
         Infectious Diseases   8516     2850     5666     33.466416
           Tobacco Addiction  10198     4369     5829     42.841734
Acute Respiratory Infections  11455     4078     7377     35.600175


---

## 7. LLM Models for Evaluation

All models run **locally via Ollama** - no data sent to external APIs.

In [73]:
import pandas as pd

models = {
    'Large Models (RTX 5090)': [
        ('DeepSeek-R1 32B', '32B', 'State-of-the-art reasoning'),
        ('Qwen 2.5 32B', '32B', 'Rivals GPT-4 on benchmarks'),
        ('Llama 3.3 70B', '70B (Q4)', 'Largest Llama, quantized'),
    ],
    'General-Purpose Models': [
        ('Llama 3.2', '3B', 'Meta baseline'),
        ('Llama 3.1 8B', '8B', 'Strong instruction-following'),
        ('Mistral 7B', '7B', 'Efficient general model'),
        ('Mixtral 8x7B', '46.7B MOE', 'Mixture of Experts'),
        ('Qwen 2.5 14B', '14B', 'Top benchmarks at size'),
        ('Gemma 2 9B', '9B', 'Google, excellent for classification'),
        ('Phi-3 Medium', '14B', 'Microsoft efficient model'),
    ],
    'Biomedical Models': [
        ('OpenBioLLM-8B', '8B', 'Outperforms GPT-3.5 on medical'),
        ('BioMistral 7B', '7B', 'Fine-tuned on PubMed Central'),
        ('Meditron 7B', '7B', 'Medical guidelines + PubMed'),
    ]
}

print("="*70)
print("LLM MODELS FOR EVALUATION (13 Models via Ollama)")
print("="*70)

for category, model_list in models.items():
    print(f"\n{category}:")
    print("-"*50)
    for name, size, desc in model_list:
        print(f"  {name:20s} | {size:12s} | {desc}")

LLM MODELS FOR EVALUATION (13 Models via Ollama)

Large Models (RTX 5090):
--------------------------------------------------
  DeepSeek-R1 32B      | 32B          | State-of-the-art reasoning
  Qwen 2.5 32B         | 32B          | Rivals GPT-4 on benchmarks
  Llama 3.3 70B        | 70B (Q4)     | Largest Llama, quantized

General-Purpose Models:
--------------------------------------------------
  Llama 3.2            | 3B           | Meta baseline
  Llama 3.1 8B         | 8B           | Strong instruction-following
  Mistral 7B           | 7B           | Efficient general model
  Mixtral 8x7B         | 46.7B MOE    | Mixture of Experts
  Qwen 2.5 14B         | 14B          | Top benchmarks at size
  Gemma 2 9B           | 9B           | Google, excellent for classification
  Phi-3 Medium         | 14B          | Microsoft efficient model

Biomedical Models:
--------------------------------------------------
  OpenBioLLM-8B        | 8B           | Outperforms GPT-3.5 on medical
  Bio

---

## 8. Evaluation Configuration

### Prompt Strategies

**Two prompt types are evaluated:**

| Strategy | Description |
|----------|-------------|
| **Zero-shot** | Direct question with 3 few-shot examples |
| **Chain-of-Thought (CoT)** | Step-by-step PICOS analysis before decision |

### Key Configuration

```python
TEMPERATURE = 0.2          # Low for deterministic outputs
MAX_TOKENS = 2048          # Space for CoT reasoning
FEW_SHOT_EXAMPLES = 3      # Calibration examples
```

### Calibration Statement

Prompts include explicit calibration:
> *"In systematic review screening, typically only 15-25% of papers meet inclusion criteria."*

This addresses the tendency of LLMs to default to "INCLUDE" for uncertain cases.

### PICOS Framework (CoT prompts)

- **P**opulation: Does the study population match?
- **I**ntervention: Is the intervention relevant?
- **C**omparison: Is there an appropriate comparator?
- **O**utcome: Are outcomes aligned with the review?
- **S**tudy design: Is the design appropriate?

---

## 9. Evaluation Metrics

### Why Standard Accuracy Isn't Enough

With **80% EXCLUDE** class imbalance, a model that predicts "EXCLUDE" for everything gets:
- **Accuracy = 80%** (looks good!)
- **Recall = 0%** (misses ALL relevant papers!)

Similarly, predicting "INCLUDE" for everything:
- **Recall = 100%** (finds all papers)
- **Specificity = 0%** (fails to reduce workload)

### Key Metrics

| Metric | Formula | Target | Why It Matters |
|--------|---------|--------|----------------|
| **Recall** | TP/(TP+FN) | ≥95% | Cannot miss relevant studies |
| **Specificity** | TN/(TN+FP) | ≥50% | Must reduce screening workload |
| **Precision** | TP/(TP+FP) | -- | Quality of INCLUDE predictions |
| **F1 Score** | Harmonic mean | -- | Balance of precision & recall |

### Practical Target

For real-world deployment:
- **Recall ≥ 95%**: Miss <5% of relevant papers
- **Specificity ≥ 50%**: Exclude at least half of irrelevant papers
- This would **halve human reviewer workload** while maintaining quality

---

## 10. Evaluation Results

In [74]:
# Check for evaluation results
RESULTS_DIR = DATA_DIR / 'results'

if RESULTS_DIR.exists():
    eval_files = sorted(RESULTS_DIR.glob('eval_*.csv'))
    print(f"Found {len(eval_files)} evaluation result files:\n")
    
    results_summary = []
    for f in eval_files:
        try:
            df = pd.read_csv(f)
            name_parts = f.stem.replace('eval_', '').split('_')
            # Extract model and prompt type
            if 'zero_shot' in f.stem:
                model = '_'.join(name_parts[:-3])
                prompt_type = 'zero_shot'
            elif 'cot' in f.stem:
                model = '_'.join(name_parts[:-2])
                prompt_type = 'cot'
            else:
                model = '_'.join(name_parts[:-1])
                prompt_type = 'unknown'
            
            # Calculate metrics if label columns exist
            if 'label' in df.columns and 'prediction' in df.columns:
                from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
                
                y_true = df['label'].astype(int)
                y_pred = df['prediction'].astype(int)
                
                recall = recall_score(y_true, y_pred, zero_division=0)
                precision = precision_score(y_true, y_pred, zero_division=0)
                f1 = f1_score(y_true, y_pred, zero_division=0)
                # Specificity
                tn = ((y_true == 0) & (y_pred == 0)).sum()
                fp = ((y_true == 0) & (y_pred == 1)).sum()
                specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
                
                results_summary.append({
                    'Model': model,
                    'Prompt': prompt_type,
                    'Records': len(df),
                    'Recall': f'{recall:.1%}',
                    'Specificity': f'{specificity:.1%}',
                    'Precision': f'{precision:.1%}',
                    'F1': f'{f1:.3f}'
                })
        except Exception as e:
            print(f"  Error reading {f.name}: {e}")
    
    if results_summary:
        results_df = pd.DataFrame(results_summary)
        print(results_df.to_string(index=False))
else:
    print("No evaluation results found yet.")
    print("Run 06_evaluate_llms.ipynb to generate results.")

Found 3 evaluation result files:

  Error reading eval_mistral_cot_20260209_021041.csv: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].
  Error reading eval_openbiollm_zero_shot_20260209_001108.csv: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].
       Model    Prompt  Records Recall Specificity Precision    F1
mistral_zero zero_shot     4089  98.2%        4.7%     21.2% 0.349


---

## 11. File Reference

### Data Files

| File | Records | Description |
|------|---------|-------------|
| `cochrane_pubmed_abstracts.csv` | 17,298 | Cochrane review metadata from PubMed |
| `cochrane_pdfs/` | 16,588 | Downloaded Cochrane PDFs (not in git) |
| `categorized_references.csv` | 629,561 | References with INCLUDE/EXCLUDE labels |
| `referenced_paper_abstracts.csv` | 47,518 | Matched papers with abstracts |
| `ground_truth_validation_dataset.csv` | 41,692 | Final validation dataset |
| `results/eval_*.csv` | varies | Model predictions per evaluation |

### Notebooks

| Notebook | Purpose | Key Output |
|----------|---------|------------|
| `00_obtain_cochrane_abstracts` | Fetch Cochrane reviews | `cochrane_pubmed_abstracts.csv` |
| `01_eda_cochrane_reviews` | Exploratory analysis | Visualizations |
| `02_fetch_cochrane_pdfs` | Download PDFs | `cochrane_pdfs/` |
| `03_extract_metadata_and_references` | PDF extraction | `categorized_references.csv` |
| `04_fetch_referenced_abstracts` | Match to PubMed | `referenced_paper_abstracts.csv` |
| `05_build_ground_truth` | Build validation set | `ground_truth_validation_dataset.csv` |
| `06_evaluate_llms` | Run LLM evaluations | `results/eval_*.csv` |
| `07_project_documentation` | This notebook | Documentation |

---

## 12. Technical Setup

### Hardware
- **GPU**: NVIDIA RTX 5090 (32GB VRAM)
- **RAM**: 32GB+ recommended
- **Storage**: ~50GB for data and models

### Software Dependencies

```bash
# Core
pip install pandas scikit-learn jupyter matplotlib plotly

# API Access
pip install biopython requests python-dotenv

# PDF Processing
pip install pymupdf

# LLM Inference
pip install ollama
```

### Environment Variables (.env)

```
NCBI_EMAIL=your.email@institution.edu
NCBI_API_KEY=your_ncbi_api_key
WILEY_TEXT_AND_DATA_MINING_TOKEN=your_wiley_token
```

### Ollama Model Setup

```powershell
# Install from https://ollama.com then:
ollama pull llama3.2
ollama pull mistral
ollama pull deepseek-r1:32b
ollama pull qwen2.5:32b
ollama pull llama3.3:70b-instruct-q4_K_M
```